In [2]:
import requests
import defs
import pandas as pd


In [3]:
session = requests.Session()

In [4]:
ins_df = pd.read_pickle("instruments.pkl")

In [5]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

Exploring the power of Data Frames

In [6]:
ins_df.shape

(68, 5)

In [7]:
ins_df.shape[0]

68

In [8]:
ins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   type         68 non-null     object
 2   displayName  68 non-null     object
 3   pipLocation  68 non-null     int64 
 4   marginRate   68 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.8+ KB


Fetch Candles from Oanda API

In [9]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

Get Candles from API Response

In [10]:
def get_candles_df(json_response):
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
            our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

Function that Saves Data Frame to File

In [11]:
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f"his_data/{pair}_{granularity}.pkl")

Function to Create Data by iterating through all symbols

In [15]:
def create_data (pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, f"HTTP Error {code}")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [16]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, "H1")
            

EUR_USD loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
EUR_GBP loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
EUR_JPY loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
EUR_CHF loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
EUR_NZD loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
EUR_CAD loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
USD_JPY loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
USD_CHF loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
USD_CAD loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
GBP_USD loaded 3 candles from 2021-06-09T09:00:00.000000000Z to 2021-06-09T09:00:00.000000000Z
GBP_JPY loaded 3 candles from 2021-06-09T09:00:00.